### **Código para reentrenar el modelo `efficientnet_b7`**
---

Hay que subir toda la carpeta `IdealistaAI` al cluster para que funcione el código. Una vez ejecutado, en la carpeta `models_generator/models` se generará el modelo `efficientnet_rakn_7.pt` reentrenado.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import os
import logging
import sys
import wandb

# Configuración de logging más detallada
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout),  # Salida a consola
        logging.FileHandler("training_log.txt"),
    ],
)


# Función para cargar los datos
def load_data(train_dir, valid_dir, batch_size=64, img_size=224):
    """
    Se encarga de cargar y preparar los datos de entrenamiento y validación para un modelo de aprendizaje profundo.
    Realiza las siguientes tareas:
    1. Aplica transformaciones a las imágenes, como redimensionamiento, normalización y conversión a tensores.
    2. Verifica la existencia de los directorios de entrenamiento y validación.
    3. Carga los conjuntos de datos desde los directorios especificados utilizando `ImageFolder`.
    4. Crea los dataloaders para los conjuntos de entrenamiento y validación, configurando el tamaño de batch, el número de trabajadores y el uso de memoria compartida.
    5. Devuelve los dataloaders y el número de clases presentes en los datos.

    Args:
    - `train_dir` (str): Ruta al directorio que contiene los datos de entrenamiento.
    - `valid_dir` (str): Ruta al directorio que contiene los datos de validación.
    - `batch_size` (int, opcional): Tamaño del batch para los dataloaders. Valor por defecto: 64.
    - `img_size` (int, opcional): Tamaño al que se redimensionarán las imágenes. Valor por defecto: 224.

    Retruns:
    - `train_loader` (DataLoader): Dataloader para el conjunto de entrenamiento.
    - `valid_loader` (DataLoader): Dataloader para el conjunto de validación.
    - `len(train_dataset.classes)` (int): Número de clases en los datos.
    """

    # Transformaciones de datos (imagenes)
    transform = torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize((img_size, img_size)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
            ),
        ]
    )

    # Verificar existencia de directorios
    if not os.path.exists(train_dir):
        raise ValueError(f"El directorio de entrenamiento no existe: {train_dir}")
    if not os.path.exists(valid_dir):
        raise ValueError(f"El directorio de validación no existe: {valid_dir}")

    # Cargar conjuntos de datos
    train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform)
    valid_dataset = torchvision.datasets.ImageFolder(valid_dir, transform=transform)

    # Crear dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=20,  # Ajusta según tus recursos
        pin_memory=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=20,  # Ajusta según tus recursos
    )

    return train_loader, valid_loader, len(train_dataset.classes)


# Función para evaluar el modelo
def evaluate(model, valid_loader, criterion):
    """
    Evalúa el rendimiento de un modelo en un conjunto de datos de validación.

    La función realiza las siguientes tareas:
    1. Cambia el modelo al modo de evaluación para desactivar el cálculo de gradientes.
    2. Itera sobre el conjunto de datos de validación para calcular la pérdida promedio y la precisión.
    3. Devuelve la pérdida promedio y la precisión del modelo en el conjunto de validación.

    Args:
    - `model` (torch.nn.Module): El modelo a evaluar.
    - `valid_loader` (torch.utils.data.DataLoader): Dataloader que contiene el conjunto de datos de validación.
    - `criterion` (torch.nn.Module): Función de pérdida utilizada para calcular la pérdida del modelo.

    Retruns:
    - `avg_loss` (float): Pérdida promedio del modelo en el conjunto de validación.
    - `accuracy` (float): Precisión del modelo en el conjunto de validación, expresada como un porcentaje.
    """

    # Establecer el modelo en modo de evaluación
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Desactivar el cálculo de gradientes
    with torch.no_grad():

        # Iterar sobre el conjunto de datos de validación
        for inputs, labels in valid_loader:

            outputs = model(inputs)  # Resultados del modelo
            loss = criterion(outputs, labels)  # Calcular la pérdida
            running_loss += loss.item()  # Acumular pérdida

            # Calcular accuracy
            _, predicted = torch.max(outputs, 1)  # Obtener la clase predicha
            correct_predictions += (predicted == labels).sum().item()  # Contar aciertos
            total_predictions += labels.size(0)  # Total de predicciones

    avg_loss = running_loss / len(valid_loader)
    accuracy = 100 * correct_predictions / total_predictions
    return avg_loss, accuracy


# Función principal de entrenamiento
def train(rank, world_size):
    """
    Entrena un modelo de aprendizaje profundo utilizando un único dispositivo.

    La función realiza las siguientes tareas:
    1. Carga los datos de entrenamiento y validación desde los directorios especificados.
    2. Configura un modelo preentrenado `efficientnet_b7` y ajusta su capa de clasificación para el número de clases en los datos.
    3. Define los hiperparámetros, como la tasa de aprendizaje y el número de épocas.
    4. Entrena el modelo en el conjunto de datos de entrenamiento, calculando la pérdida y la precisión en cada batch.
    5. Evalúa el modelo en el conjunto de validación al final de cada época.
    6. Guarda el modelo entrenado en un archivo al finalizar el entrenamiento.

    Args:
    - `rank` (int): Identificador del dispositivo o proceso que realiza el entrenamiento.
    - `world_size` (int): Número total de dispositivos o procesos utilizados para el entrenamiento (en este caso, siempre es 1).

    Retruns:
    - `None`: La función no retorna ningún valor, pero guarda el modelo entrenado en un archivo.
    """

    try:
        print(f"Comenzando entrenamiento en rank {rank}")

        # Configuración del conjunto de datos
        train_dir = "../dataset/training"
        valid_dir = "../dataset/validation"

        # Cargar datos
        train_loader, valid_loader, num_classes = load_data(
            train_dir,
            valid_dir,
            batch_size=64,  # Usamos un solo dispositivo
            img_size=224,  # [futuro] A lo mejor para otros modelos hay que cargar las imagenes con otro tamaño
        )

        print(f"Número de clases: {num_classes}")
        print(f"Longitud del conjunto de entrenamiento: {len(train_loader.dataset)}")

        # Preparar modelo
        model = torchvision.models.efficientnet_b7(
            weights="DEFAULT"
        )  # [futuro] hacer algo para que cambie el modelo directamente
        #
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

        # Configuración de hiperparámetros
        learning_rate = 1e-4 * world_size
        epochs = 7

        # Optimizador y criterio
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()

        print("Comenzando bucle de entrenamiento")

        # Entrenamiento
        for epoch in range(epochs):
            print(f"Época {epoch+1}/{epochs}")

            model.train()
            running_loss = 0.0
            correct_predictions = 0
            total_predictions = 0

            # Entrenamiento por cada batch
            for batch_idx, (inputs, labels) in enumerate(train_loader):
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # Calcular accuracy para el conjunto de entrenamiento
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_predictions += labels.size(0)

                # Imprimir cada 10 batches
                if batch_idx % 10 == 0:
                    accuracy = 100 * correct_predictions / total_predictions
                    print(
                        f"Época {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}, Train Accuracy: {accuracy:.2f}%"
                    )

            # Calcular loss y accuracy para el conjunto de validación
            val_loss, val_accuracy = evaluate(model, valid_loader, criterion)
            print(
                f"Época {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%"
            )

        # Guardar modelo
        os.makedirs("../models", exist_ok=True)
        torch.save(model.state_dict(), f"../models/efficientnet_rank_{rank}.pt")
        print(f"Modelo guardado exitosamente para rank {rank}")

    except Exception as e:
        print(f"Error durante el entrenamiento: {e}")
        import traceback

        traceback.print_exc()


def main():
    # Solo usamos 1 dispositivo
    world_size = 1

    # Entrenamiento sin DDP
    train(rank=7, world_size=world_size)


# Llamar a la función principal
if __name__ == "__main__":
    main()

Comenzando entrenamiento en rank 7
Número de clases: 15
Longitud del conjunto de entrenamiento: 2985


c:\Users\pablo\anaconda3\envs\dl_lab\Lib\site-packages\torch\utils\data\dataloader.py:558: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyboardInterrupt: 